<a href="https://colab.research.google.com/github/vnprr/spotify_predictor/blob/main/songs_spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Programowanie/SpotifyPredicterRefactored")

In [ ]:
import matplotlib.pyplot as plt
import nltk
import pandas as pd
import seaborn as sns
import string
from joblib import dump
from keras.layers import Dense
from keras.models import Sequential
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from tqdm import tqdm

In [ ]:
from songs_spotify import load_artists, load_spotify_songs, load_joined_songs, get_Xy, train_test_split

# Data preparation

## Artists last_fm

In [ ]:
# load last fm data
artists = load_artists()

artists

## Spotify songs

In [ ]:
# load spotify data
spotify_songs = load_spotify_songs()

spotify_songs

##Join sets

In [ ]:
songs_joined = load_joined_songs(spotify_songs, artists)

## Compute all songs xy

In [ ]:
X, y = get_Xy(songs_joined)

##Train/Test Split

In [ ]:
#XY
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1337)

##Training

##Ridge

In [ ]:
ridge = RidgeCV(alphas=[0.001, 0.0001, 0.01, 1, 0.1])
ridge.fit(X_train, y_train)

In [ ]:
ridge.score(X_train, y_train)

In [ ]:
ridge.score(X_test, y_test)

##SVR  - deprecated

In [ ]:
#model tested, too much time, awful performance

# parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[0.1, 1, 10]}
# svr = SVR()
# model = GridSearchCV(svr, parameters, n_jobs = -1, cv = 3, verbose=True)

# opt
# parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 1]}
# svr = SVR()
# model = GridSearchCV(svr, parameters, n_jobs=2, cv=3, verbose=True)


# model.fit(X_train, y_train)

In [ ]:
# model.score(X_train, y_train)

In [ ]:
# model.score(X_test, y_test)

##K-NN

In [ ]:
parameters = {'n_neighbors': (3, 5, 10, 15, 25, 50)}
knn = KNeighborsRegressor()
model_knn = GridSearchCV(knn, parameters, n_jobs = -1, cv = 3, verbose=True)
model_knn.fit(X_train, y_train)

In [ ]:
model_knn.score(X_train, y_train)

In [ ]:
model_knn.score(X_test, y_test)

##random forest

In [ ]:
# test / train split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1337)
# RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=400, max_depth=10, random_state=42)
# train
rf_model.fit(X_train, y_train)
# predict on test
y_pred = rf_model.predict(X_test)



In [ ]:
rf_model.score(X_train, y_train)

In [ ]:
rf_model.score(X_test, y_test)

In [ ]:
plt.figure(figsize = (7, 7))

# draw random forest
sns.kdeplot(x=y_test, y=y_pred)
sns.regplot(x=y_test, y=y_pred, scatter=False)

##random boosting (best score)

In [ ]:
gb_model = GradientBoostingRegressor(n_estimators=500, learning_rate=0.1, max_depth=5, random_state=42)

# Train model
gb_model.fit(X_train, y_train)

# Predict on X_test
y_pred_gb = gb_model.predict(X_test)



In [ ]:
gb_model.score(X_train, y_train)

In [ ]:
gb_model.score(X_test, y_test)

In [ ]:
sns.kdeplot(x=y_test, y=y_pred_gb)

In [ ]:
gbmodel_path = "models/gb_model.joblib"
os.makedirs(os.path.dirname(gbmodel_path), exist_ok = True)
dump(gb_model, gbmodel_path)

## Neural Networks

In [ ]:
# architecture
nn_model = Sequential()
nn_model.add(Dense(256, activation="relu", input_shape=(X_train.shape[1],)))
nn_model.add(Dense(128, activation="relu"))
# ^ #128, 128, 50: tr 0.5822491450527092, 0.5244461518283765
nn_model.add(Dense(128, activation="relu"))
# ^ #128, 128, 128, 50: tr 0.5964326981226203 te 0.541574869925696
#nn_model.add(Dense(128, activation="relu"))
# ^ 128, 128, 128, 128, 50: tr 0.6420181327529579, te 0.5339284515859302
nn_model.add(Dense(1))

# compilation
nn_model.compile(optimizer="rmsprop", loss="mean_absolute_error")

# training
history = nn_model.fit(x=X_train.values, y=y_train.values, epochs=50, batch_size=32, validation_split=0.2, verbose=True)



In [ ]:
#rate
y_pred = nn_model.predict(X_test)
y_train_pred = nn_model.predict(X_train)

print(r2_score(y_train, y_train_pred))
print(r2_score(y_test, y_pred))
print(nn_model.summary())

In [ ]:
#overfitting - jak spada walidacyjny to testowy też spada razem z nim, tu jest lekko
plt.plot(history.history['loss'], label="train")
plt.plot(history.history['val_loss'], label="val")

plt.legend()

In [ ]:
plt.figure(figsize = (7, 7))
sns.kdeplot(x=y_test, y=y_pred.flatten())
sns.regplot(x=y_test, y=y_pred, scatter=False)